# Silver NEAR Data NLP WordEmbeddings v05

## Dependencies and Packages

### Download Google Word Embeddings

In [ ]:
!pip install gdown --ignore-install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached filelock-3.11.0-py3-none-any.whl (10.0 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached soupsieve-2.4-py3-none-any.whl (37 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached charset_normalizer-3.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (199 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)


In [ ]:
!gdown '1VPwziUXxRukY8qvbLaVfITav9JBjFXKH'

Downloading...
From (uriginal): https://drive.google.com/uc?id=1VPwziUXxRukY8qvbLaVfITav9JBjFXKH
From (redirected): https://drive.google.com/uc?id=1VPwziUXxRukY8qvbLaVfITav9JBjFXKH&confirm=t&uuid=e3122646-2e9d-4abc-8437-0a8c39207a8e
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:45<00:00, 36.5MB/s]


In [ ]:
!ls -la

total 1690384
drwxr-xr-x 1 root root       4096 Apr 13 14:10 .
drwxr-xr-x 1 root root       4096 Apr 13 13:55 ..
drwxr-xr-x 4 root root       4096 Apr 11 13:32 .config
-rw-r--r-- 1 root root 1647046227 Apr 13 14:10 GoogleNews-vectors-negative300.bin.gz
-rw-r--r-- 1 root root   83886080 Apr 13 14:09 GoogleNews-vectors-negative300.bin.gztdkxo078tmp
drwxr-xr-x 1 root root       4096 Apr 11 13:33 sample_data


In [ ]:
!gunzip GoogleNews-vectors-negative300.bin

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.3 MB/s eta 0:00:00
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)


In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=becaa88e5c157943cd667ddae7dc50b0a16619aa28bead8a275a11b1c26f2b31
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.1 MB/s eta 0:00:00


### Import Packages

In [ ]:
import glob
import json
import os
import re

from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np
pd.options.display.max_columns = None

## Preprocessing

In [ ]:
import re

import contractions
import demoji
import nltk
import tqdm


def normalize_document(doc):
    doc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', ' ', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()

    return doc

def normalize_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        norm_doc = normalize_document(doc)
        norm_docs.append(norm_doc)

    return norm_docs

# Working with silver_near_social_txs_parsed table

In [ ]:
path = '/content/drive/MyDrive/DataScienceData/Pagoda-NEAR/'
data_path = path + 'silver_posts.csv'

In [ ]:
silver_posts = pd.read_csv(data_path)
silver_posts

,signer_id,post_text,block_timestamp
0,harucoinlove.near,I will be giving a talk in the Twitter space o...,1675090010763536263
1,ea5f61c0e53d3580117f3c5e137c19fb2fc190c89d5c24...,"Hello people, what's going on?",1675105556950351297
2,toolipse.near,https://www.mintbase.xyz/meta/toolipse.mintbas...,1675076830691131033
3,microchipgnu.near,I'm flying tomorrow and can't think of an easy...,1675118864026317834
4,microchipgnu.near,Another day sharing updates on the chain.,1675097550040763134
...,...,...,...
7392,fe8436ddaccee2819d3a425d29a382bbe74f229447fba5...,what about the C++ community on this social ne...,1674303473512570373
7393,67d4ee978da716b8138b9f22562969d4337a7a84672414...,Scam memes😹😹😹,1674310790442328443
7394,orkhan001.near,greenery in the markets,1674313749432546291
7395,porfire.near,Wait this all gets stored on-chain?,1674323236102837974


In [ ]:
df = silver_posts
df = df.drop_duplicates()
df = df.dropna(subset="post_text").copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6031 entries, 0 to 7390
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   signer_id        6031 non-null   object
 1   post_text        6031 non-null   object
 2   block_timestamp  6031 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 188.5+ KB


In [ ]:
df['clean_text'] = df['post_text'].apply(lambda x: demoji.replace_with_desc(x, sep=" "))
df['clean_text'] = normalize_corpus(df['clean_text'].values)

100%|██████████| 6031/6031 [00:00<00:00, 20191.56it/s]


In [ ]:
df[df["clean_text"].str.contains("http")]

,signer_id,post_text,block_timestamp,clean_text
7299,mr27.near,"### March 19\n\nAh, March 19. A day of many in...",1679180414183962013,march ah march a day of many inventions both s...


In [ ]:
posts = df['clean_text']

In [ ]:
posts.str.find("http")

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
7386   -1
7387   -1
7388   -1
7389   -1
7390   -1
Name: clean_text, Length: 6031, dtype: int64

In [ ]:
type(posts)

pandas.core.series.Series

In [ ]:
sentences = list(posts)

In [ ]:
sentences

['i will be giving a talk in the twitter space on the following dates about multi chain including near avax cosmos near mina dogechain kcs algorand cardano tron aptos and will talk about the latest information about them jst gmt',
 'hello people what is going on',
 '',
 'i am flying tomorrow and cannot think of an easy way to test widget development without internet this might be a step in that direction',
 'another day sharing updates on the chain',
 'near is all you need',
 'what is next',
 'hi near friends',
 'introducing the ecosystem influencer roundtable posted originally on in the interest of transparency and improving collaboration on shared goals across the near ecosystem near foundation is formalizing a new initiative for this year the ecosystem influencer roundtable designed to increase communication and increase accountability between major ecosystem projects pagoda and the foundation the ecosystem roundtable has started meeting weekly in the purpose of the roundtable group

## Use Sentence Transformers and Generate Corpus Embeddings


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
# model = 'roberta-large-nli-stsb-mean-tokens' # deprecated
model = 'all-mpnet-base-v2'
embedder = SentenceTransformer(model)

In [ ]:
corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True) # ! runs around 30min on CPU !

In [ ]:
import pickle


filename = path + f'models/corpus_embeddings_{model}.pickle'

with open(filename, 'wb') as f:
    pickle.dump(corpus_embeddings, f)
    

print(f"The pickled file is saved at {filename}")

The pickled file is saved at /content/drive/MyDrive/DataScienceData/Pagoda-NEAR/models/corpus_embeddings_all-mpnet-base-v2.pickle


In [ ]:
filename = path + f'models/corpus_embeddings_{model}.pickle'
with open(filename, 'rb') as f:
    corpus_embeddings = pickle.load(f)

# h5_filename = path + 'models/corpus_embeddings.h5'
# with h5py.File(h5_filename, 'r') as f:
#     corpus_embeddings = f['embeddings'][:]

In [ ]:
corpus_embeddings.shape

torch.Size([6031, 768])

In [ ]:
from sentence_transformers import util
import torch

def return_similar_sentences(query, model_embedder, corpus_embeddings, top_k):
    query_embedding = model_embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    top_results = torch.topk(cos_scores, k=top_k)

    top_n_sentences = []
    for score, idx in zip(top_results[0], top_results[1]):
        post_id = df.iloc[int(idx)]
        username = df.iloc[int(idx)]['signer_id']
        similarity_score = float(score)
        sentence = sentences[int(idx)]
        top_n_sentences.append((sentence, similarity_score, post_id, username))
    
    return top_n_sentences

### Perform Semantic Search on Sample Questions to get Similar Queries from the Corpus

In [ ]:
custom_sentence = "I just minted my first NFT on the chain! Lookie-lookie! I feel great!"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('one of the nfts i minted when i just opened my near wallet rolling on the floor laughing',
  0.659323513507843,
  signer_id                                            kemiiisola.near
  post_text          One of the nfts I minted when I just opened my...
  block_timestamp                                  1677946474655503478
  clean_text         one of the nfts i minted when i just opened my...
  Name: 4069, dtype: object,
  'kemiiisola.near'),
 ('your nft has been minted your nft has been successfully minted on our marketplace',
  0.6494114398956299,
  signer_id                                          phanquynhchi.near
  post_text          Your NFT has been minted!\n\nYour NFT has been...
  block_timestamp                                  1681229544160175617
  clean_text         your nft has been minted your nft has been suc...
  Name: 1288, dtype: object,
  'phanquynhchi.near'),
 ('i got in on an nft tonight wish me luck',
  0.5950894355773926,
  signer_id                          

In [ ]:
custom_sentence = "Gah, what a boring place, anyone likes to watch some cartoons?"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('cartoon',
  0.5269625186920166,
  signer_id           azmainfiqe001.near
  post_text                      Cartoon
  block_timestamp    1677954577588226861
  clean_text                     cartoon
  Name: 4306, dtype: object,
  'azmainfiqe001.near'),
 ('are you guys having fun here',
  0.36522841453552246,
  signer_id          d63c4537c813fc561dae7d5829daac40b140d7a249036b...
  post_text                             Are you guys having fun here..
  block_timestamp                                  1678003072411460301
  clean_text                              are you guys having fun here
  Name: 598, dtype: object,
  'd63c4537c813fc561dae7d5829daac40b140d7a249036b5053564667dcdf8b90'),
 ('boredape',
  0.34845393896102905,
  signer_id              fayezzarai.near
  post_text                     Boredape
  block_timestamp    1677970393843493087
  clean_text                    boredape
  Name: 4752, dtype: object,
  'fayezzarai.near'),
 ('aiiiigghhhttt what is good over here',
  0.342579424

In [ ]:
custom_sentence = "Flipping the script while being the director, frens"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('writing gives you the illusion of control and then you realize it is just an illusion that people are going to bring their own stuff into it david sedaris',
  0.3277348279953003,
  signer_id                                              maicodon.near
  post_text          "Writing gives you the illusion of control, an...
  block_timestamp                                  1679217990941627847
  clean_text         writing gives you the illusion of control and ...
  Name: 6370, dtype: object,
  'maicodon.near'),
 ('guess the scene',
  0.326050341129303,
  signer_id                gjezza14.near
  post_text              Guess the scene
  block_timestamp    1678030686306673637
  clean_text             guess the scene
  Name: 915, dtype: object,
  'gjezza14.near'),
 ('this is future fr',
  0.3124326467514038,
  signer_id          493572653104b3a5b5061c3f48fe28e7c020f23dfd445f...
  post_text                                          This is Future fr
  block_timestamp                           

In [ ]:
custom_sentence = "I found this quote by Alexander Hamilton, very interesting!"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('february it is february and i am traveling through the snow covered hills of new jersey with general washington on a mission to convince local militia to join the continental army after an inspection of the troops and an impassioned speech in which i declare my commitment to our because i hear an old man exclaim this is the alexander hamilton i have heard about we would better listen to what he has to say though i am blushing from the praise i remind the men that our because is just and i am honored to be part of it little did i know at the time that just a few years later i would be devoting my life to preserving the union and creating a new nation under the constitution it is moments like these that always remind me of the importance of our revolution and the great responsibility we have to bring it to fruition',
  0.5115879774093628,
  signer_id                                            littlelion.near
  post_text          ### February 15\n\nIt's February 15, 1781 and ...
  bloc

In [ ]:
custom_sentence = "Best way to create realistic mockups in #photoshop"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('free ai tools to generate images artificial intelligence has revolutionized the way we create and consume content one area where ai has made significant strides is in generating images there are now several free ai tools available that can help you create stunning images without any design skills one such tool is deep dream generator it uses a neural network to generate images based on the user s inputs the user can choose from a variety of styles and tweak the settings to get the desired output the results are often surreal and dreamlike hence the name another popular tool is canva s magic resize feature canva is a graphic design platform that offers a range of templates and design elements its magic resize feature uses ai to automatically resize images to fit various social media platforms this saves time and effort for content creators who need to create multiple versions of the same image finally there is artbreeder which allows users to generate images by combining different el

In [ ]:
custom_sentence = "Why Don’t We Just Wait Here For A Little While, See What Happens?"

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('waiting for something big',
  0.4588552713394165,
  signer_id                    liveairdrop.near
  post_text          Waiting for something big 
  block_timestamp           1678345541767130584
  clean_text          waiting for something big
  Name: 1666, dtype: object,
  'liveairdrop.near'),
 ('while we wait for life life passes',
  0.4054034948348999,
  signer_id                                abhishek3.near
  post_text          While we wait for life, life passes.
  block_timestamp                     1678006564545699854
  clean_text           while we wait for life life passes
  Name: 666, dtype: object,
  'abhishek3.near'),
 ('i do not know exactly how it going until now',
  0.377551794052124,
  signer_id                              truongngochuyenbk13.near
  post_text          I don't know exactly how it going until now!
  block_timestamp                             1678198667409923760
  clean_text         i do not know exactly how it going until now
  Name: 1604, dtype: obje

In [ ]:
custom_sentence = sentences[480]

query_sentence = custom_sentence
return_similar_sentences(
    query=query_sentence, 
    model_embedder=embedder, 
    corpus_embeddings=corpus_embeddings,
    top_k=5
)

[('i am very excited about this project every one must be join this project this project value may up rocket',
  1.0,
  signer_id                                              rama2001.near
  post_text          I am very excited about this project.Every one...
  block_timestamp                                  1677998780438140930
  clean_text         i am very excited about this project every one...
  Name: 512, dtype: object,
  'rama2001.near'),
 ('very good project iam interested',
  0.7884117364883423,
  signer_id                         hasan31060628.near
  post_text          Very good project iam interested 
  block_timestamp                  1677952849410427517
  clean_text          very good project iam interested
  Name: 4158, dtype: object,
  'hasan31060628.near'),
 ('this a very promising project sir i hope your team become a successful and your projects will go to the moon we always support you',
  0.7285440564155579,
  signer_id          0f8753f902fac608ebf43bd231ba2e9df679d